# AI Financial Advisor

This notebook demonstrates how to build an AI-powered financial assistant.


# Step 1: Install required libraries

We'll be using the following Python libraries:

- **gspread** and **oauth2client** for interacting with Google Sheets.

- **pandas** for data manipulation.
- **openai** for LLM interactions.
- **streamlit** for the interactive UI



In [ ]:
!pip install gspread oauth2client pandas openai streamlit

# Step 2: Google Sheets Setup

## 2.1 Create a Google Sheet Template

Create a new Google Sheet with the following columns:
- Date
- Description
- Category (initially empty)
- Income/Expense
- Amount

## 2.2 Set Up Google Cloud Project and Enable Google Sheets API

1. Go to the Google Cloud Console.
2. Create a new project.
3. Navigate to APIs & Services > Library.
4. Search for Google Sheets API and enable it.


## 2.3 Create Service Account Credentials

1. In the Google Cloud Console, go to APIs & Services > Credentials.
2. Click Create Credentials > Service Account.
3. Follow the prompts and create a JSON key file.
4. Download the JSON credentials file to your project directory.

## 2.4 Share the Google Sheet with the Service Account
- In your Google Sheet, click Share.
- Share the sheet with the service account email (found in your JSON key under client_email).

# Step 3: Data Extraction

## 3.1 Authenticate and Connect to Google Sheets

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Define the scope
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]

# Add credentials to the account
creds = ServiceAccountCredentials.from_json_keyfile_name('/path/to/credentials.json', scope)

# Authorize the clientsheet
client = gspread.authorize(creds)

## 3.2 Load Data into Pandas DataFrame

In [ ]:
import pandas as pd

# Open the Google Sheet
sheet = client.open('Personal Transactions Data').sheet1

# Get all records
data = sheet.get_all_records()

# Convert to DataFrame
df = pd.DataFrame(data)

df

,Date,Description,Category,Income/Expense,Amount
0,2024-09-01,Salary,Income,Income,5000.00
1,2024-09-02,Grocery shopping at Coles,Groceries,Expense,150.50
2,2024-09-03,Electricity bill,Utilities,Expense,75.80
3,2024-09-04,Netflix Subscription,Entertainment,Expense,12.99
4,2024-09-05,Dinner at Olive Garden,Dining Out,Expense,45.20
5,2024-09-07,Freelance Project,Income,Income,1200.00
6,2024-09-10,Uber Ride,Transportation,Expense,23.45
7,2024-09-12,Gym Membership,,Expense,35.00
8,2024-09-15,Rent Payment,,Expense,1200.00
9,2024-09-18,Office Supplies,Miscellaneous,Expense\t,50.00


# Step 4: Categorizing Transactions with LLM

## 4.1 Prepare Transaction Descriptions for Categorization

In [ ]:
# Extract descriptions without categories
descriptions = df[df['Category'] == '']['Description'].tolist()
descriptions

['Gym Membership', 'Rent Payment', 'Donation to Charity']

## 4.2 Define a Function to Categorize Transactions

In [ ]:
from openai import OpenAI

# Setup OPENAI_API_KEY
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")


openai_client = OpenAI()

def categorize_transaction(description):
    completion = openai_client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[
            {"role": "system", "content": "You are a helpful financial assistant."},
            {
                "role": "user",
                "content": f"""
                  Categorize the following transaction: '{description}'.
                  Possible categories are: Groceries, Utilities, Rent, Entertainment, Transportation, Dining Out, Miscellaneous, Health & Fitness, Housing, Investments, Insurance, Charity, and Income.
                  Your answer should contain the category name only without explanations."""
            }
        ]
    )
    category = completion.choices[0].message.content.strip()
    return category

In [ ]:
print(categorize_transaction("Rent Payment"))

Rent


## 4.3 Apply Function to Categorize Multiple Transactions

In [ ]:
# Apply the categorization function to uncategorized transactions
for index, row in df[df['Category'] == ''].iterrows():
    category = categorize_transaction(row['Description'])
    df.at[index, 'Category'] = category

# Update the Google Sheet with the new categories
for index, row in df.iterrows():
    sheet.update_cell(index + 2, df.columns.get_loc('Category') + 1, row['Category'])

# Step 5 Generating Financial Summaries with GPT-4o-mini

We will create the function to generate financial summaries using **gpt-4o-mini**


## 5.1 Generate Monthly Financial Summary

In [ ]:
def generate_financial_summary(total_income, total_spent, category_spending):
  completion = openai_client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[
        {"role": "system", "content": "You are a helpful financial assistant."},
        {
          "role": "user",
          "content": f"Generate a financial summary based on the following data:\n"
                      f"Total Income: ${total_income:.2f}\n"
                      f"Total Expenses: ${total_spent:.2f}\n"
                      f"Spending by category: {category_spending.to_dict()}."
                      "You should output in plaintext only."
        }
      ]
  )
  summary = completion.choices[0].message.content.strip()
  return summary

## 5.2 Call the Summary Function


In [ ]:
total_spent = df[df['Income/Expense'] == 'Expense']['Amount'].sum()
total_income = df[df['Income/Expense'] == 'Income']['Amount'].sum()
category_spending = df[df['Income/Expense'] == 'Expense'].groupby('Category')['Amount'].sum()

financial_summary = generate_financial_summary(total_income, total_spent, category_spending)
print(financial_summary)

Financial Summary:

Total Income: $6415.00  
Total Expenses: $2012.74  

Net Income (Income - Expenses): $4402.26

Spending by Category:  
- Charity: $100.00  
- Dining Out: $49.70  
- Entertainment: $37.99  
- Groceries: $370.80  
- Health & Fitness: $35.00  
- Insurance: $120.00  
- Rent: $1200.00  
- Transportation: $23.45  
- Utilities: $75.80  

Total Spending: $2012.74  
Remaining Funds after Expenses: $4402.26


# Step 6: Personalized Financial Advice

We will create the function to generate financial advice

## 6.1 Generate Financial Advice Based on Transaction Data

In [ ]:
def generate_personalized_advice(df):
  largest_expense = df[df['Income/Expense'] == 'Expense'].nlargest(1, 'Amount')

  completion = openai_client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[
        {"role": "system", "content": "You are a helpful financial assistant."},
        {
          "role": "user",
          "content": f"Based on the following transaction history, provide personalized financial advice:\n"
                      f"Largest expense: {largest_expense['Description'].values[0]} (${largest_expense['Amount'].values[0]:.2f})\n"
                      f"Categories where spending is increasing."
                      f"You should output in plaintext only."
        }
      ]
  )
  advice = completion.choices[0].message.content.strip()
  return advice

## 6.2 Call the Advice Function

In [ ]:
financial_advice = generate_personalized_advice(df)
print(financial_advice)

Based on your transaction history and the indication that your largest expense is your rent payment ($1200.00), here are some personalized financial advice points to consider:

1. **Budget Review**: Assess your entire monthly budget, particularly the categories where spending is increasing. Identify specific areas where you can cut back, such as dining out, entertainment, or shopping, to allocate more funds toward essential expenses or savings.

2. **Necessity Check**: Reevaluate your increasing spending categories. Determine if the expenses are necessary or if there are alternatives that could help reduce costs, like cooking at home instead of dining out.

3. **Goal Setting**: Set clear financial goals to help motivate your spending habits. This could include saving for an emergency fund, a travel fund, or paying off debt.

4. **Rent Consideration**: Since rent is your largest expense, consider if it’s feasible to negotiate your rent with your landlord, or if relocating to a more affo

## 6.3 Function to convert markdown to text

The output from GPT model may be in markdown format, so we need to parse MD before saving in PDF reports. We will use **Beautifulsoup** to do this

In [ ]:
!pip install beautifulsoup4

In [ ]:
from markdown import markdown
import re

def markdown_to_html(markdown_string):
    """Converts a markdown string to HTML, including support for tables."""

    # Convert markdown to HTML, enabling table extensions
    html = markdown(markdown_string)

    return html

financial_summary = markdown_to_html(financial_summary)
print(financial_summary)

<p>Financial Summary:</p>
<p>Total Income: $6415.00<br />
Total Expenses: $2012.74  </p>
<p>Net Income (Income - Expenses): $4402.26</p>
<p>Spending by Category:<br />
- Charity: $100.00<br />
- Dining Out: $49.70<br />
- Entertainment: $37.99<br />
- Groceries: $370.80<br />
- Health &amp; Fitness: $35.00<br />
- Insurance: $120.00<br />
- Rent: $1200.00<br />
- Transportation: $23.45<br />
- Utilities: $75.80  </p>
<p>Total Spending: $2012.74<br />
Remaining Funds after Expenses: $4402.26</p>


In [ ]:
financial_advice = markdown_to_html(financial_advice)
print(financial_advice)

<p>Based on your transaction history and the indication that your largest expense is your rent payment ($1200.00), here are some personalized financial advice points to consider:</p>
<ol>
<li>
<p><strong>Budget Review</strong>: Assess your entire monthly budget, particularly the categories where spending is increasing. Identify specific areas where you can cut back, such as dining out, entertainment, or shopping, to allocate more funds toward essential expenses or savings.</p>
</li>
<li>
<p><strong>Necessity Check</strong>: Reevaluate your increasing spending categories. Determine if the expenses are necessary or if there are alternatives that could help reduce costs, like cooking at home instead of dining out.</p>
</li>
<li>
<p><strong>Goal Setting</strong>: Set clear financial goals to help motivate your spending habits. This could include saving for an emergency fund, a travel fund, or paying off debt.</p>
</li>
<li>
<p><strong>Rent Consideration</strong>: Since rent is your largest

# Step 7: Report Automation

## 7.1 Generate a PDF Report

We can use libraries like **pdfkit** to generate PDF reports.

In [ ]:
!pip install pdfkit
!apt-get update
!apt-get install -y wkhtmltopdf

import pdfkit

# Configure pdfkit to point to our installed wkhtmltopdf
config = pdfkit.configuration(wkhtmltopdf='/usr/bin/wkhtmltopdf')

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy Release
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 257 kB in 1s (184 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to pro

In [ ]:
import pdfkit

def create_pdf_report(summary, advice):
    html_content = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="UTF-8">
        <title>Monthly Financial Report</title>
        <style>
            body {{
                font-family: 'Arial', sans-serif;
            }}
        </style>
    </head>
    <body>
        <h1 style="text-align: center;">Monthly Financial Report</h1>

        <h2>Summary</h2>
        {summary}

        <h2>Advice</h2>
        {advice}
    </body>
    </html>
    """

    options = {
        'encoding': 'UTF-8',
    }

    pdfkit.from_string(html_content, 'financial_report.pdf', options=options)


create_pdf_report(financial_summary, financial_advice)

# Notes

- Currently in our prompt we instruct the model to output only in plaintext but we can have problems that can affect the format of pdf file we want to save